In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import pickle
from shapely import Point
from shapely.ops import nearest_points
from rtree import index
from tqdm import tqdm

from shapely.geometry import Point, Polygon, LineString,MultiLineString

In [4]:
# import Node_Block.pkl

with open(r'intermediate_files/Node_Block.pkl', 'rb') as f:
    Node_Block = pickle.load(f)

In [5]:
Node_Block

,Node,BlockNumber,BlockBoundary
0,0,060378002062007,"MULTIPOLYGON (((-118.68372 34.164316, -118.683..."
1,1,060379201061055,"MULTIPOLYGON (((-118.681741 34.444332, -118.68..."
2,2,060378005041027,"MULTIPOLYGON (((-118.685613 34.036776, -118.68..."
3,3,060378002021012,"MULTIPOLYGON (((-118.68486 34.084816, -118.684..."
4,4,060379201061079,"MULTIPOLYGON (((-118.707669 34.436242, -118.70..."
...,...,...,...
180082,162810,060371132351008,"MULTIPOLYGON (((-118.645234 34.238084, -118.64..."
180083,3196,060371132351004,"MULTIPOLYGON (((-118.636906 34.2404, -118.6363..."
180084,162813,060371132351007,"MULTIPOLYGON (((-118.643995 34.240384, -118.64..."
180085,162814,060371132351007,"MULTIPOLYGON (((-118.643995 34.240384, -118.64..."


In [6]:
# number of unique values in Node_Block['Node']

len(Node_Block['Node'].unique())

180087

In [7]:
with open(r'nodes_edges_ucla_big_graph.pickle', 'rb') as handle:
    B_matrix_sliced,B_matrix_str_sliced,nodes_coordinates_array = pickle.load(handle)

In [8]:
# dimensions of nodes_coordinates_array

nodes_coordinates_array.shape

(180087, 2)

In [10]:
Origin_Destination = pd.read_csv('lodes_od_data/ca_od_main_JT00_2021.csv')

In [11]:
Initial_Origin_Destination_Count = len(Origin_Destination)

In [12]:
# Create dictionary for block and node pairings (identified from BlockDetermination.ipynb)
block_node_dict = {}

In [13]:
for _, row in Node_Block.iterrows():
    node_id = row['Node']
    block_id = row['BlockNumber']
    # block_coord = row['BlockBoundary']
    if int(block_id) not in block_node_dict:
        block_node_dict[int(block_id)] = []
    block_node_dict[int(block_id)].append(node_id)

Origin_Destination_Node_Added = Origin_Destination.copy()

In [14]:
Origin_Destination_Node_Added

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,60014001001003,60014004002014,1,0,0,1,0,0,1,0,0,1,20231016
1,60014001001003,60014017002013,1,1,0,0,0,0,1,0,0,1,20231016
2,60014001001003,60014045011006,1,0,1,0,0,0,1,0,0,1,20231016
3,60014001001003,60014047001005,1,0,0,1,0,0,1,0,0,1,20231016
4,60014001001003,60014053012003,1,0,1,0,0,0,1,0,0,1,20231016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041046,61150411021047,61150411021023,1,0,0,1,0,1,0,0,0,1,20231016
15041047,61150411021047,61150411021024,1,0,0,1,0,1,0,1,0,0,20231016
15041048,61150411021047,61150411021025,1,0,0,1,0,1,0,0,0,1,20231016
15041049,61150411021047,61150411021031,1,0,0,1,0,1,0,0,0,1,20231016


In [15]:
# drop createdate column

Origin_Destination_Node_Added = Origin_Destination_Node_Added.drop(columns=['createdate'])

In [16]:
# Map nodes to work and home blocks
Origin_Destination_Node_Added['w_node_id'] = Origin_Destination_Node_Added['w_geocode'].map(block_node_dict)
Origin_Destination_Node_Added['h_node_id'] = Origin_Destination_Node_Added['h_geocode'].map(block_node_dict)

In [17]:
Origin_Destination_Node_Added

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_node_id,h_node_id
0,60014001001003,60014004002014,1,0,0,1,0,0,1,0,0,1,NaN,NaN
1,60014001001003,60014017002013,1,1,0,0,0,0,1,0,0,1,NaN,NaN
2,60014001001003,60014045011006,1,0,1,0,0,0,1,0,0,1,NaN,NaN
3,60014001001003,60014047001005,1,0,0,1,0,0,1,0,0,1,NaN,NaN
4,60014001001003,60014053012003,1,0,1,0,0,0,1,0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041046,61150411021047,61150411021023,1,0,0,1,0,1,0,0,0,1,NaN,NaN
15041047,61150411021047,61150411021024,1,0,0,1,0,1,0,1,0,0,NaN,NaN
15041048,61150411021047,61150411021025,1,0,0,1,0,1,0,0,0,1,NaN,NaN
15041049,61150411021047,61150411021031,1,0,0,1,0,1,0,0,0,1,NaN,NaN


In [18]:
# how many rows have w_node_id and h_node_id as NaN

Origin_Destination_Node_Added['w_node_id'].isna().sum() / len(Origin_Destination_Node_Added)

0.8265825971868588

In [19]:
# Remove rows where the node columns are missing (i.e. the block is not in the node dictionary)
Origin_Destination_Node_Added = Origin_Destination_Node_Added[Origin_Destination_Node_Added['w_node_id'].notnull()]
Origin_Destination_Node_Added = Origin_Destination_Node_Added[Origin_Destination_Node_Added['h_node_id'].notnull()]

In [20]:
Origin_Destination_Node_Added.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1425945 entries, 1959609 to 6053876
Data columns (total 14 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   w_geocode  1425945 non-null  int64 
 1   h_geocode  1425945 non-null  int64 
 2   S000       1425945 non-null  int64 
 3   SA01       1425945 non-null  int64 
 4   SA02       1425945 non-null  int64 
 5   SA03       1425945 non-null  int64 
 6   SE01       1425945 non-null  int64 
 7   SE02       1425945 non-null  int64 
 8   SE03       1425945 non-null  int64 
 9   SI01       1425945 non-null  int64 
 10  SI02       1425945 non-null  int64 
 11  SI03       1425945 non-null  int64 
 12  w_node_id  1425945 non-null  object
 13  h_node_id  1425945 non-null  object
dtypes: int64(12), object(2)
memory usage: 498.4 MB


In [21]:
# store S000, SA01, SA02, SA03, SE01, SE02, SE03, SI01, SI02, SI03 columns in a separate dataframe

Origin_Destination_Node_Added_Work = Origin_Destination_Node_Added[['w_geocode', 'h_geocode', 'S000', 'SA01', 'SA02', 'SA03', 'SE01', 'SE02', 'SE03', 'SI01', 'SI02', 'SI03']]

# remove the columns from the original dataframe

# Origin_Destination_Node_Added = Origin_Destination_Node_Added.drop(columns=['S000', 'SA01', 'SA02', 'SA03', 'SE01', 'SE02', 'SE03', 'SI01', 'SI02', 'SI03'])

In [22]:
Origin_Destination_Node_Added

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_node_id,h_node_id
1959609,60371011101000,60371011102004,1,0,1,0,0,1,0,0,0,1,"[118106, 118114, 118119]","[112096, 112556]"
1959610,60371011101000,60371021072013,1,0,1,0,0,1,0,0,0,1,"[118106, 118114, 118119]","[81706, 81855, 175590]"
1959611,60371011101000,60371286011008,1,0,0,1,0,1,0,0,0,1,"[118106, 118114, 118119]",[42821]
1959612,60371011101000,60371920012009,1,0,1,0,0,0,1,0,0,1,"[118106, 118114, 118119]",[87006]
1959614,60371011101001,60371011103006,1,0,1,0,0,1,0,0,0,1,"[117479, 117637]",[111269]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6053870,60379800391003,60379200291006,1,0,1,0,0,0,1,0,0,1,"[82902, 82983, 83045, 83094, 83117, 83143, 831...","[30108, 30214, 30418, 30625, 30684]"
6053872,60379800391003,60379200411018,1,0,1,0,0,0,1,0,0,1,"[82902, 82983, 83045, 83094, 83117, 83143, 831...","[43840, 43844, 173855, 173856]"
6053874,60379800391003,60379203222004,1,0,0,1,0,1,0,0,0,1,"[82902, 82983, 83045, 83094, 83117, 83143, 831...","[17372, 17397, 17417, 17502, 17579, 17758, 178..."
6053875,60379800391003,60379203312000,1,0,0,1,0,0,1,0,0,1,"[82902, 82983, 83045, 83094, 83117, 83143, 831...","[16142, 16230, 16358, 16418, 16512, 16543, 165..."


In [23]:
# print memory of Origin_Destination_Node_Added

Origin_Destination_Node_Added.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1425945 entries, 1959609 to 6053876
Data columns (total 14 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   w_geocode  1425945 non-null  int64 
 1   h_geocode  1425945 non-null  int64 
 2   S000       1425945 non-null  int64 
 3   SA01       1425945 non-null  int64 
 4   SA02       1425945 non-null  int64 
 5   SA03       1425945 non-null  int64 
 6   SE01       1425945 non-null  int64 
 7   SE02       1425945 non-null  int64 
 8   SE03       1425945 non-null  int64 
 9   SI01       1425945 non-null  int64 
 10  SI02       1425945 non-null  int64 
 11  SI03       1425945 non-null  int64 
 12  w_node_id  1425945 non-null  object
 13  h_node_id  1425945 non-null  object
dtypes: int64(12), object(2)
memory usage: 498.4 MB


In [24]:
# convert Origin_Destination_Node_Added to array

Origin_Destination_Node_Added_array = Origin_Destination_Node_Added.to_numpy()

In [25]:
Origin_Destination_Node_Added_array

array([[60371011101000, 60371011102004, 1, ..., 1,
        list([118106, 118114, 118119]), list([112096, 112556])],
       [60371011101000, 60371021072013, 1, ..., 1,
        list([118106, 118114, 118119]), list([81706, 81855, 175590])],
       [60371011101000, 60371286011008, 1, ..., 1,
        list([118106, 118114, 118119]), list([42821])],
       ...,
       [60379800391003, 60379203222004, 1, ..., 1,
        list([82902, 82983, 83045, 83094, 83117, 83143, 83146, 83244, 83444, 83480, 83481, 83734, 84128, 84590, 85532, 85594, 85660, 85727, 85781, 85788, 85864, 86440, 86492, 88671, 166731, 166732, 176234]),
        list([17372, 17397, 17417, 17502, 17579, 17758, 17864, 17869, 17954, 17955, 17987, 17988, 17989, 18033, 18041, 18122, 18126, 18244, 18245, 18246, 18267, 18276, 18308, 18320, 18503, 18508, 18513, 18523, 18544, 18547, 172515])],
       [60379800391003, 60379203312000, 1, ..., 1,
        list([82902, 82983, 83045, 83094, 83117, 83143, 83146, 83244, 83444, 83480, 83481, 83734, 

In [26]:
# extract only columns 0 and 1 from Origin_Destination_Node_Added_array

Block_to_Block_Pairs = Origin_Destination_Node_Added_array[:, 0:2]

In [27]:
Block_to_Block_Pairs

array([[60371011101000, 60371011102004],
       [60371011101000, 60371021072013],
       [60371011101000, 60371286011008],
       ...,
       [60379800391003, 60379203222004],
       [60379800391003, 60379203312000],
       [60379800391003, 60379203431009]], dtype=object)

In [28]:
# check if each row in Block_to_Block_Pairs array is a unique pair

Block_to_Block_Pairs = Block_to_Block_Pairs.astype(float)  # or int, depending on your data
unique_rows = np.unique(Block_to_Block_Pairs, axis=0)
is_all_unique = len(Block_to_Block_Pairs) == len(unique_rows)

In [29]:
is_all_unique

True

In [30]:
# extract only columns 2 to 11 from Origin_Destination_Node_Added_array

LODES_info_to_be_adjusted = Origin_Destination_Node_Added_array[:, 2:12]

In [31]:
LODES_info_to_be_adjusted

array([[1, 0, 1, ..., 0, 0, 1],
       [1, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1]], dtype=object)

In [32]:
len(LODES_info_to_be_adjusted)

1425945

In [33]:
# Origin_Destination_Node_Added_array[:, 12:14]

block_pairing_count_list = []

for nodes in Origin_Destination_Node_Added_array[:, 12:14]:
    block_pairing_count = len(nodes[0]) * len(nodes[1])
    block_pairing_count_list.append(block_pairing_count)


In [34]:
len(block_pairing_count_list)

1425945

In [35]:
for i in range(len(LODES_info_to_be_adjusted)):
    LODES_info_to_be_adjusted[i] = LODES_info_to_be_adjusted[i] / block_pairing_count_list[i]

In [36]:
LODES_adjusted = LODES_info_to_be_adjusted

In [37]:
Node_to_Node_lists = Origin_Destination_Node_Added_array[:, 12:14]

In [38]:
Node_to_Node_lists

array([[list([118106, 118114, 118119]), list([112096, 112556])],
       [list([118106, 118114, 118119]), list([81706, 81855, 175590])],
       [list([118106, 118114, 118119]), list([42821])],
       ...,
       [list([82902, 82983, 83045, 83094, 83117, 83143, 83146, 83244, 83444, 83480, 83481, 83734, 84128, 84590, 85532, 85594, 85660, 85727, 85781, 85788, 85864, 86440, 86492, 88671, 166731, 166732, 176234]),
        list([17372, 17397, 17417, 17502, 17579, 17758, 17864, 17869, 17954, 17955, 17987, 17988, 17989, 18033, 18041, 18122, 18126, 18244, 18245, 18246, 18267, 18276, 18308, 18320, 18503, 18508, 18513, 18523, 18544, 18547, 172515])],
       [list([82902, 82983, 83045, 83094, 83117, 83143, 83146, 83244, 83444, 83480, 83481, 83734, 84128, 84590, 85532, 85594, 85660, 85727, 85781, 85788, 85864, 86440, 86492, 88671, 166731, 166732, 176234]),
        list([16142, 16230, 16358, 16418, 16512, 16543, 16547, 16548, 16587, 16630, 17281, 18024, 18242, 172279, 172311, 172312, 172323, 172335, 

In [51]:
Origin_Destination_Node_Added_array[:,:12][0]

array([60371011101000, 60371011102004, 0.16666666666666666, 0.0,
       0.16666666666666666, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0,
       0.16666666666666666], dtype=object)

In [39]:
from itertools import product

Node_to_Node_pairs = []

for nodes in Node_to_Node_lists:
    Node_to_Node_pairs.extend(product(nodes[0], nodes[1]))

In [40]:
len(Node_to_Node_pairs)

88813000

In [53]:
Node_to_Node_pairs

[(118106, 112096),
 (118106, 112556),
 (118114, 112096),
 (118114, 112556),
 (118119, 112096),
 (118119, 112556),
 (118106, 81706),
 (118106, 81855),
 (118106, 175590),
 (118114, 81706),
 (118114, 81855),
 (118114, 175590),
 (118119, 81706),
 (118119, 81855),
 (118119, 175590),
 (118106, 42821),
 (118114, 42821),
 (118119, 42821),
 (118106, 87006),
 (118114, 87006),
 (118119, 87006),
 (117479, 111269),
 (117637, 111269),
 (117479, 122742),
 (117479, 122881),
 (117479, 123527),
 (117479, 123906),
 (117479, 176640),
 (117479, 176642),
 (117637, 122742),
 (117637, 122881),
 (117637, 123527),
 (117637, 123906),
 (117637, 176640),
 (117637, 176642),
 (116490, 111274),
 (116490, 111280),
 (116490, 111287),
 (116490, 111288),
 (116490, 176785),
 (116496, 111274),
 (116496, 111280),
 (116496, 111287),
 (116496, 111288),
 (116496, 176785),
 (116889, 111274),
 (116889, 111280),
 (116889, 111287),
 (116889, 111288),
 (116889, 176785),
 (116490, 55061),
 (116496, 55061),
 (116889, 55061),
 (116490

In [41]:
# Export Node_to_Node_pairs to a pickle file

with open('intermediate_files/Node_to_Node_pairs.pickle', 'wb') as handle:
    pickle.dump(Node_to_Node_pairs, handle, protocol=pickle.HIGHEST_PROTOCOL)

    

In [42]:
# Export Origin_Destination_Node_Added_array to a pickle file

with open('intermediate_files/Origin_Destination_Node_Added_array.pickle', 'wb') as handle:
    pickle.dump(Origin_Destination_Node_Added_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [52]:
# Export block_pairing_count_list to a pickle file

with open('intermediate_files/block_pairing_count_list.pickle', 'wb') as handle:
    pickle.dump(block_pairing_count_list, handle, protocol=pickle.HIGHEST_PROTOCOL)